#### Hartree-Fock in Pytorch

In [3]:
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState
import numpy as np

from typing import Dict
import scipy
from NSMFermions.qml_models import AdaptVQEFermiHubbard
from NSMFermions.qml_utils.train import Fit
from NSMFermions.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.hartree_fock_library import HFEnergyFunctional,HFEnergyFunctionalNuclear,HFEnergyFunctionalUnitary,build_fock_matrix,transform_integrals
from scipy.sparse import lil_matrix
import torch
import torch.nn as nn
import torch.optim as optim

file_name='data/cki'


SPS=SingleParticleState(file_name=file_name)

In [2]:
twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:01<00:00, 11.67it/s]


In [4]:
nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{8}$Be'

In [5]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])

NSMHamiltonian.get_twobody_interaction_optimized(twobody_dict=twobody_matrix)

NSMHamiltonian.get_hamiltonian()

egs,psigs=NSMHamiltonian.get_spectrum(n_states=10)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psigs,basis=NSMHamiltonian.basis))
#print('j_value=',J2Class.j_value(psi0))
print('dimension=',NSMHamiltonian.hamiltonian.shape[0])

/home/ecosta/miniconda3/envs/annealing-nsm-env/lib/python3.10/site-packages/numba/typed/typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to uint64. Precision may be lost.
  return d[key]


size= 12 6
Building two-body operator with 1424 terms...


100%|██████████| 1424/1424 [00:01<00:00, 1357.39it/s]

✅ Two-body operator built: shape=(51, 51), nnz=1155
[-30.29539461 -26.88369692 -19.0041005  -15.86699708 -15.31298892
 -14.49860676 -13.61172347 -13.41558684 -12.80630663 -12.62779145]
total_m= [0.00000000e+00 0.00000000e+00 1.51788304e-18 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.77555756e-17 5.20417043e-18
 1.38777878e-17 2.08166817e-17]
dimension= 51


Hartree-Fock method

In [6]:
hf = HFEnergyFunctionalUnitary(torch.diag(torch.tensor(SPS.energies)), twobody_matrix, nparticles_a+nparticles_b)
opt = torch.optim.Adam(hf.parameters(), lr=1e-2)

v_ext=torch.diag(torch.tensor(SPS.energies)).detach().numpy()

for it in range(1000):
    opt.zero_grad()
    E = hf()
    E.backward()
    opt.step()

    if it % 50 == 0:
        print(it, E.item())

0 -8.018729762837612
50 -14.266796381928376
100 -21.882680104861294
150 -25.401957274650613
200 -25.68802581679233
250 -25.689024454797824
300 -25.689032354278527
350 -25.6890323574768
400 -25.6890323575773
450 -25.689032357578427
500 -25.689032357578444
550 -25.68903235757844
600 -25.68903235757844
650 -25.68903235757843
700 -25.68903235757843
750 -25.68903235757844
800 -25.689032357578448
850 -25.689032357578434
900 -25.689032357578434
950 -25.689032357578427


In [7]:
v_matrix=np.zeros((size_a+size_b,size_a+size_b,size_a+size_b,size_a+size_b))
for key in twobody_matrix.keys():
    a,b,c,d=key
    v_matrix[a,b,c,d]=twobody_matrix[key]

fock_operator=build_fock_matrix(h_mat=v_ext,V_tensor=v_matrix,rho=hf.rho)

eps, U_can = np.linalg.eigh(fock_operator)   # columns of U_can are eigenvectors
print(eps)
# U_can maps canonical index -> old basis index as columns; we want U matrix with rows=new index, cols=old basis
# Here U_can has shape (M,M) as columns are eigenvectors in old basis; define U = U_can.T
U = U_can.T.copy()
# Transform integrals
h_p, V_p, Vbar_p = transform_integrals(U, v_ext, v_matrix)

[-6.42493241 -6.42493241 -6.42493241 -6.42493241 -0.6498448  -0.6498448
 -0.6498448  -0.6498448   1.06353834  1.06353834  1.06353834  1.06353834]


In [8]:
new_energies=np.diag(v_ext)
print(new_energies)

new_twobody_matrix={}
for a in range(size_a+size_b):
    for b in range(size_a+size_b):
        for c in range(size_a+size_b):
            for d in range(size_a+size_b):
                if np.abs(V_p[a,b,c,d])>=10**-7:
                    new_twobody_matrix[(a,b,c,d)]=V_p[a,b,c,d]

[1.63 1.63 1.63 1.63 2.27 2.27 1.63 1.63 1.63 1.63 2.27 2.27]


In [11]:
print((new_twobody_matrix))

{(0, 1, 0, 1): np.float64(-5.6152120965814865), (0, 1, 0, 2): np.float64(0.05426841090782962), (0, 1, 0, 3): np.float64(0.13648320471070038), (0, 1, 0, 4): np.float64(0.02336800478170449), (0, 1, 0, 5): np.float64(-0.012612549831252984), (0, 1, 0, 6): np.float64(-0.20376339300801025), (0, 1, 0, 7): np.float64(0.015192924744033397), (0, 1, 0, 8): np.float64(-0.2445490407454952), (0, 1, 0, 9): np.float64(-0.04846192972887208), (0, 1, 0, 10): np.float64(-0.05731819991975015), (0, 1, 0, 11): np.float64(0.1377946748663818), (0, 1, 1, 0): np.float64(5.615212096581487), (0, 1, 1, 2): np.float64(0.015031099601950641), (0, 1, 1, 3): np.float64(-0.7453639118393608), (0, 1, 1, 4): np.float64(-0.04312426259472067), (0, 1, 1, 5): np.float64(0.2372013866127068), (0, 1, 1, 6): np.float64(-0.03246018566664215), (0, 1, 1, 7): np.float64(0.3000944063239453), (0, 1, 1, 8): np.float64(0.05467930894679375), (0, 1, 1, 9): np.float64(-0.016029324356565178), (0, 1, 1, 10): np.float64(-0.07562886566195165), (0

In [12]:
NSMFockHamiltonian=FermiHubbardHamiltonian(size_a=size_a+size_b,size_b=0,nparticles_a=nparticles_a+nparticles_b,nparticles_b=0)
NSMFockHamiltonian.get_external_potential(new_energies)
NSMFockHamiltonian.get_twobody_interaction_optimized(new_twobody_matrix)
NSMFockHamiltonian.get_hamiltonian()
egs,psigshf=NSMFockHamiltonian.get_spectrum(10)

print(egs)

Building two-body operator with 15888 terms...


  0%|          | 0/15888 [00:00<?, ?it/s]

100%|██████████| 15888/15888 [00:01<00:00, 15634.73it/s]

✅ Two-body operator built: shape=(495, 495), nnz=94055
[-30.25708824 -27.04947471 -27.01562053 -26.81953862 -26.66186695
 -26.2092432  -19.17668332 -19.17546202 -19.13357941 -19.03019773]


In [18]:
from NSMFermions.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp

QPC=QuasiParticlesConverterOnlynnpp()
QPC.initialize_shell(state_encoding=SPS.state_encoding)
QPC.get_the_basis_matrix_transformation(NSMHamiltonian.basis)


QPCHF=QuasiParticlesConverterOnlynnpp()
QPCHF.initialize_shell(state_encoding=SPS.state_encoding)
QPCHF.get_the_basis_matrix_transformation(NSMFockHamiltonian.basis)

new_couples=[]
for i in range(0,size_a+size_b//2,1):
    new_couples.append([i,i+6])
    
    
QPCHF.couples=new_couples






In [19]:
print(QPC.quasiparticle_basis.shape)
hamiltonian_Q=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
egsQ,psi_Q=eigsh(hamiltonian_Q,k=8,which='SA')

hamiltonian_QHF=QPCHF.particles2quasiparticles @ NSMFockHamiltonian.hamiltonian @ QPCHF.particles2quasiparticles.T
egsQnp,psi_Qnp=eigsh(hamiltonian_QHF,k=10,which='SA')

print(egsQnp,egsQ)






print('\n')

#print(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))
fs=[]
for i in range(egsQnp.shape[0]):
    psi_Qnp2particle=QPCHF.particles2quasiparticles.T @ psi_Qnp[:,i]
    psi_Qnp2particle/=np.linalg.norm(psi_Qnp2particle)
    print(psigshf[:,0].dot(psi_Qnp2particle.conj())*np.conjugate(psigshf[:,0].dot(psi_Qnp2particle.conj())))#,j_values[i])
    fs.append(psigshf[:,0].dot(psi_Qnp2particle.conj())*np.conjugate(psigshf[:,0].dot(psi_Qnp2particle.conj())))
print('\n')
print(np.sum(fs),'\n')
fs=[]
for i in range(egsQ.shape[0]):
    psi_Q2particle=QPC.particles2quasiparticles.T @ psi_Q[:,i]
    psi_Q2particle/=np.linalg.norm(psi_Q2particle)
    
    print(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))#,j_values[0])
    fs.append(psigs[:,0].dot(psi_Q2particle.conj())*np.conjugate(psigs[:,0].dot(psi_Q2particle.conj())))
print('\n')
print(np.sum(fs))


(9, 6)
[-26.32340029 -11.64251053 -11.35858784 -11.29851126  -8.89439196
  -8.66336268  -8.47855318  -6.48045291  -6.20159967  -5.08205126] [-20.3041178  -17.43478828 -12.52467658 -10.67469442  -6.97037031
  -5.557375    -1.44209672  -0.34163842]


0.3166497124118008
2.7738538119564287e-05
0.00026142657744267765
0.00018962210274032652
0.0002086096938731601
0.00010022245962888636
5.669027579394455e-06
0.0005105034756096961
2.9623100836225164e-05
1.6986066119540876e-05


0.3180001134537503 

0.5758878400309745
1.7806493797144825e-32
3.702485718623336e-33
0.009593520332546544
0.0008179919153660388
3.6955792282986574e-34
6.933347799794049e-33
1.1393013975070348e-34


0.5862993522788871
